# Aprendizado Profundo - UFMG


## Regulizadores

Neste código iremos analisar como funcionam os regularizadores e como eles são usados para evitar *overfitting*, fenônemo que faz com o modelo não generalize bem em outros dataset além do usando durante o treino.

# *Overfitting*

Redes neurais são muito flexíveis porque não se limitam a ver cada atributo a ser aprendido individualmente. Em vez disso, elas podem aprender interações entre os atributos. Por causa disso, mesmo quando temos apenas um pequeno número de atributos, as redes neurais profundas são capazes de chegar ao *overfitting*, um cenário onde o modelo aprende a classificar muito bem (as vezes, perfeitamente) as instâncias de treino, porém não generaliza para outras instâncias não vista (como são os casos de amostras do conjunto de validação ou teste).

Para evitar esse cenário, algumas técnicas foram propostas para evitar o *overfitting*.

**Como introduzido, nesta aula prática, implementaremos e testaremos duas técnicas para evitar o *overfitting*: *Dropout* e *Weight Decay***

Esse pequeno bloco de código abaixo é usado somente para instalar o MXNet para CUDA 10. Execute esse bloco somente uma vez e ignore possíveis erros levantados durante a instalação.

**ATENÇÃO: a alteração deste bloco pode implicar em problemas na execução dos blocos restantes!**

In [0]:
!pip install mxnet-cu100

# imports basicos
import time, os, sys
import mxnet as mx
from mxnet import autograd, gluon, init, nd
from mxnet.gluon import loss as gloss, nn, utils as gutils, data as gdata

# Tenta encontrar GPU
def try_gpu():
    try:
        ctx = mx.gpu()
        _ = nd.zeros((1,), ctx=ctx)
    except mx.base.MXNetError:
        ctx = mx.cpu()
    return ctx

ctx = try_gpu()
ctx

     |████████████████████████████████| 488.3MB 29kB/s 
     |████████████████████████████████| 13.8MB 38.8MB/s 
ERROR: spacy 2.1.4 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
ERROR: imgaug 0.2.9 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
ERROR: fastai 1.0.52 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: blis 0.2.4 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
  Found existing installation: numpy 1.16.4
    Uninstalling numpy-1.16.4:
      Successfully uninstalled numpy-1.16.4


## *Dropout*

*Dropout* é uma das formas mais interessantes de regularizar sua rede neural. 
A ideia do *Droupout* é simples: durante o passo de *Forward*, alguns neurônios são aleatoriamente "desligados", ou seja, são zerados, e não são utilizados em nenhum processamento.
Em cada passo do *Forward*, neurônios diferentes são "desligados" aleatoriamente, de acordo com uma probabilide pré-definida.
Lembrem-se que esse processo só acontece durante o treino.
Durante o teste, *Dropout* não tem nenhuma ação e todos os neurônios são usados para gerar o resultado fina.

Formalmente, suponha um neurônio com ativação $h$ e um *Dropout* com probabilide $p$ (de zerar ou "desligar" o neurônio).
Logo, essa técnica irá "desligar" a ativação desse neurônio com probabilidade $p$ ou reescala-la baseado na probabilidade de essa unidade de processamento permanecer ativa (isto é, $1-p$):

$$
\begin{aligned}
h' =
\begin{cases}
    0 & \text{ com probabilidade } p \\
    \frac{h}{1-p} & \text{ caso contrário}
\end{cases}
\end{aligned}
$$

Tal método é interessante e chamou a atenção do mundo acadêmico por ser muito simples de implementar e poder impulsar significativamente o desempenho do modelo.

### Implementação

Em frameworks atuais (como no MxNet, TensorFlow, e PyTorch), para utilizar os benefícios do Dropout basta adicionar a camada homônima (passando como argumento a probabilidade de desligamento dos neurônios) durante a construção da arquitetura.

**Um exemplo é mostrado abaixo utilizando o framework MxNet.**

Durante o treino, a camada *Dropout* irá "desligar" aleatoriamente algumas saídas da camada anterior (ou equivalentemente, as entradas para a camada subsequente) de acordo com a probabilidade especificada.

Quando o MXNet não está no modo de treinamento, a camada *Dropout* simplesmente passa os dados sem fazer nenhum "desligamento".

In [0]:
## carregando dados

# código para carregar o dataset do MNIST
# http://yann.lecun.com/exdb/mnist/
def load_data_mnist(batch_size, resize=None, root=os.path.join(
        '~', '.mxnet', 'datasets', 'mnist')):
    """Download the MNIST dataset and then load into memory."""
    root = os.path.expanduser(root)
    transformer = []
    if resize:
        transformer += [gdata.vision.transforms.Resize(resize)]
    transformer += [gdata.vision.transforms.ToTensor()]
    transformer = gdata.vision.transforms.Compose(transformer)

    mnist_train = gdata.vision.MNIST(root=root, train=True)
    mnist_test = gdata.vision.MNIST(root=root, train=False)
    num_workers = 0 if sys.platform.startswith('win32') else 4

    train_iter = gdata.DataLoader(mnist_train.transform_first(transformer),
                                  batch_size, shuffle=True,
                                  num_workers=num_workers)
    test_iter = gdata.DataLoader(mnist_test.transform_first(transformer),
                                 batch_size, shuffle=False,
                                 num_workers=num_workers)
    return train_iter, test_iter

# código para carregar o dataset do Fashion-MNIST
# https://github.com/zalandoresearch/fashion-mnist
def load_data_fashion_mnist(batch_size, resize=None, root=os.path.join(
        '~', '.mxnet', 'datasets', 'fashion-mnist')):
    """Download the Fashion-MNIST dataset and then load into memory."""
    root = os.path.expanduser(root)
    transformer = []
    if resize:
        transformer += [gdata.vision.transforms.Resize(resize)]
    transformer += [gdata.vision.transforms.ToTensor()]
    transformer = gdata.vision.transforms.Compose(transformer)

    mnist_train = gdata.vision.FashionMNIST(root=root, train=True)
    mnist_test = gdata.vision.FashionMNIST(root=root, train=False)
    num_workers = 0 if sys.platform.startswith('win32') else 4

    train_iter = gdata.DataLoader(mnist_train.transform_first(transformer),
                                  batch_size, shuffle=True,
                                  num_workers=num_workers)
    test_iter = gdata.DataLoader(mnist_test.transform_first(transformer),
                                 batch_size, shuffle=False,
                                 num_workers=num_workers)
    return train_iter, test_iter

In [0]:
# funções básicas
def _get_batch(batch, ctx):
    """Return features and labels on ctx."""
    features, labels = batch
    if labels.dtype != features.dtype:
        labels = labels.astype(features.dtype)
    return (gutils.split_and_load(features, ctx),
            gutils.split_and_load(labels, ctx), features.shape[0])

# Função usada para calcular acurácia
def evaluate_accuracy(data_iter, net, loss, ctx=[mx.cpu()]):
    """Evaluate accuracy of a model on the given data set."""
    if isinstance(ctx, mx.Context):
        ctx = [ctx]
    acc_sum, n, l = nd.array([0]), 0, 0
    for batch in data_iter:
        features, labels, _ = _get_batch(batch, ctx)
        for X, y in zip(features, labels):
            # X, y = X.as_in_context(ctx), y.as_in_context(ctx)
            y = y.astype('float32')
            y_hat = net(X)
            l += loss(y_hat, y).sum()
            acc_sum += (y_hat.argmax(axis=1) == y).sum().copyto(mx.cpu())
            n += y.size
        acc_sum.wait_to_read()
    return acc_sum.asscalar() / n, l.asscalar() / n
  
# Função usada no treinamento e validação da rede
def train_validate(net, train_iter, test_iter, batch_size, trainer, loss, ctx,
                   num_epochs):
    print('training on', ctx)
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            X, y = X.as_in_context(ctx), y.as_in_context(ctx)
            with autograd.record():
                y_hat = net(X)
                l = loss(y_hat, y).sum()
            l.backward()
            trainer.step(batch_size)
            y = y.astype('float32')
            train_l_sum += l.asscalar()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().asscalar()
            n += y.size
        test_acc, test_loss = evaluate_accuracy(test_iter, net, loss, ctx)
        print('epoch %d, train loss %.4f, train acc %.3f, test loss %.4f, '
              'test acc %.3f, time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_loss, 
                 test_acc, time.time() - start))

gpu(0)

In [0]:
# parâmetros: número de epochs, learning rate (ou taxa de aprendizado), e 
# tamanho do batch
num_epochs, lr, batch_size = 10, 0.5, 256

# rede simples somente com perceptrons e camadas densamente conectadas
net = nn.Sequential()
net.add(nn.Dense(256, activation="relu"),  # camada densamente conectada
        nn.Dropout(0.2),                   # dropout com 20% de probabilidade de desligar os neurônios
        nn.Dense(256, activation="relu"),  # camada densamente conectada
        nn.Dropout(0.5),                   # dropout com 50% de probabilidade de desligar os neurônios
        nn.Dense(10))                      # camada densamente conectada para classificação
net.initialize(init.Normal(sigma=0.01), ctx=ctx)

# função de custo (ou loss)
loss = gloss.SoftmaxCrossEntropyLoss()

# carregamento do dado: fashion mnist
train_iter, test_iter = load_data_fashion_mnist(batch_size)

# trainer do gluon
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})

# treinamento e validação via MXNet
train_validate(net, train_iter, test_iter, batch_size, trainer, loss, 
               ctx, num_epochs)

training on gpu(0)
epoch 1, train loss 1.1607, train acc 0.553, test loss 0.5975, test acc 0.770, time 3.9 sec
epoch 2, train loss 0.5867, train acc 0.784, test loss 0.4665, test acc 0.827, time 4.3 sec
epoch 3, train loss 0.4984, train acc 0.819, test loss 0.4051, test acc 0.854, time 5.1 sec
epoch 4, train loss 0.4557, train acc 0.835, test loss 0.3840, test acc 0.860, time 4.2 sec
epoch 5, train loss 0.4253, train acc 0.843, test loss 0.3873, test acc 0.860, time 4.4 sec
epoch 6, train loss 0.4001, train acc 0.854, test loss 0.3579, test acc 0.869, time 4.4 sec
epoch 7, train loss 0.3840, train acc 0.860, test loss 0.3409, test acc 0.874, time 4.1 sec
epoch 8, train loss 0.3745, train acc 0.863, test loss 0.3623, test acc 0.868, time 4.2 sec
epoch 9, train loss 0.3630, train acc 0.867, test loss 0.3451, test acc 0.876, time 4.1 sec
epoch 10, train loss 0.3476, train acc 0.872, test loss 0.3368, test acc 0.874, time 4.3 sec


## *Weight Decay*

*Weight Decay* (comumente chamado regularização *L2*), é uma das técnicas mais utilizadas para regularizar modelos paramétricos de aprendizado de máquina.
A intuição básica por trás do *Weight Decay* é a noção de que entre todas as funções $f$, a função $f=0$ é a mais simples. Intuitivamente, podemos medir funções pela sua proximidade a zero. Mas quão devemos medir a distância entre uma função e zero? Não há uma resposta correta. De fato, ramos inteiros da matemática são dedicados a responder a esta questão.

Para nossos propósitos atuais, uma interpretação muito simples será suficiente: vamos considerar uma função linear $f(\mathbf{x}) = \mathbf{w}^\top \mathbf{x}$ é simples se o seu vetor de peso $\mathbf{w}$ for pequeno. Podemos medir isso via norma $||\mathbf{w}||^2$. Uma maneira de manter o vetor de peso pequeno é adicionar sua norma como um termo de penalidade ao problema de minimizar a função de perda (ou *loss*). Assim, nós substituímos nosso objetivo original, *minimizar o erro de previsão nos rótulos de treinamento*, com novo objetivo, *minimizar o erro de previsão e o termo de penalidade*. Agora, se o vetor de peso se tornar muito grande, nosso algoritmo de aprendizagem vai encontrar mais lucro minimizando a norma $||\mathbf{w}||^2$ do que minimizando o erro de treinamento. 

Tecnicamente, para uma função de custo qualquer $\mathcal{L}$, a adição do novo termo de penalidade (ou *weight decay*) acontece da seguinte forma:

$$\mathcal{L}(\mathbf{w}, b) + \frac{\lambda}{2} \|\boldsymbol{w}\|^2$$

Esse parâmetro não negativo $\lambda \geq 0$ dita a quantidade de regularização. Para $\lambda = 0$, recuperamos nossa função de perda original, enquanto para $\lambda > 0 $ garantimos que os pesos $\mathbf{w}$ não crescerão demais.

### Implementação

Em frameworks atuais (como no MxNet, TensorFlow, e PyTorch), *Weight Decay* pode ser facilmente agregado à função de custo durante a construção do modelo.

**Um exemplo é mostrado abaixo utilizando o framework MxNet.**

In [0]:
# parâmetros: número de epochs, learning rate (ou taxa de aprendizado), 
# tamanho do batch, e valor de weight decay
num_epochs, lr, batch_size, weight_decay = 10, 0.5, 256, 0.005

# rede simples somente com perceptrons e camadas densamente conectadas
net = nn.Sequential()
net.add(nn.Dense(256, activation="relu"),  # camada densamente conectada
        nn.Dense(256, activation="relu"),  # camada densamente conectada
        nn.Dense(10))                      # camada densamente conectada para classificação
net.initialize(init.Normal(sigma=0.01), ctx=ctx)

# função de custo (ou loss)
loss = gloss.SoftmaxCrossEntropyLoss()

# carregamento do dado: mnist
train_iter, test_iter = load_data_mnist(batch_size)

# trainer do gluon
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr, 
                                                      'wd': weight_decay})

# treinamento e validação via MXNet
train_validate(net, train_iter, test_iter, batch_size, trainer, loss, 
               ctx, num_epochs)

training on gpu(0)
epoch 1, train loss 1.0333, train acc 0.638, test loss 0.2954, test acc 0.912, time 4.3 sec
epoch 2, train loss 0.2932, train acc 0.913, test loss 0.1991, test acc 0.944, time 4.3 sec
epoch 3, train loss 0.2499, train acc 0.929, test loss 0.1928, test acc 0.947, time 5.3 sec
epoch 4, train loss 0.2015, train acc 0.943, test loss 0.1540, test acc 0.956, time 4.2 sec
epoch 5, train loss 0.1911, train acc 0.947, test loss 0.1476, test acc 0.959, time 4.2 sec
epoch 6, train loss 0.2293, train acc 0.936, test loss 0.1498, test acc 0.959, time 4.3 sec
epoch 7, train loss 0.1761, train acc 0.951, test loss 0.1555, test acc 0.957, time 4.2 sec
epoch 8, train loss 0.1724, train acc 0.952, test loss 0.2367, test acc 0.931, time 4.2 sec
epoch 9, train loss 0.1610, train acc 0.956, test loss 0.1333, test acc 0.963, time 4.1 sec
epoch 10, train loss 0.1973, train acc 0.945, test loss 0.1472, test acc 0.961, time 4.2 sec


## MNIST - *Overfitting* and regularizadores

Agora usaremos um dataset específico juntamente com um rede mais produnda para tentar mostrar o efeito de *overfitting* e entender como as técnicas de regularização aprendidas podem ser usadas para resolver esse problema.

O modelo implementado abaixo usa o dataset MNIST treinando uma rede com quatro camadas que não usa nenhum método de regularização.
Note a diferença entre o *loss* e a acurácia de treino e teste.
Um resultado onde o *loss* do teste é relativamente maior que o treino (caso do exemplo abaixo) indica que um (neste caso, princípio de) *overfitting* está acontecendo.
Use as técnicas vistas nesta aula prática para tratar esse problema.

Especificamente, implemente:

1. Uma versão dessa arquitetura com *Dropout*. Teste diferentes valores de probabilidade de forma a diminuir o *overfitting*.
2. Uma versão desse modelo com *Weight Decay*. Teste diferentes valores de $\lambda$ de forma a diminuir o *overfitting*.
3. Uma versão que combina os dois métodos de regularização aprendidos. Talvez seja necessário testar diferentes valores para a probabilidade do *Dropout* e do $\lambda$.

In [0]:
# parâmetros: número de epochs, learning rate (ou taxa de aprendizado), e 
# tamanho do batch
num_epochs, lr, batch_size = 20, 0.5, 256

# rede simples somente com perceptrons e camadas densamente conectadas
net = nn.Sequential()
net.add(nn.Dense(256, activation="relu"),
        nn.Dense(128, activation="relu"),
        nn.Dense(64, activation="relu"),
        nn.Dense(10))
net.initialize(init.Normal(sigma=0.01), ctx=ctx)

# função de custo (ou loss)
loss = gloss.SoftmaxCrossEntropyLoss()

# carregamento do dado: mnist
train_iter, test_iter = load_data_mnist(batch_size)

# trainer do gluon
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})

# treinamento e validação via MXNet
train_validate(net, train_iter, test_iter, batch_size, trainer, loss, 
               ctx, num_epochs)

training on gpu(0)
epoch 1, train loss 2.3017, train acc 0.112, test loss 2.3008, test acc 0.114, time 4.1 sec
epoch 2, train loss 2.0558, train acc 0.205, test loss 1.0226, test acc 0.567, time 4.3 sec
epoch 3, train loss 0.5507, train acc 0.824, test loss 0.1968, test acc 0.945, time 4.3 sec
epoch 4, train loss 0.1845, train acc 0.946, test loss 0.1503, test acc 0.953, time 4.5 sec
epoch 5, train loss 0.1201, train acc 0.964, test loss 0.1170, test acc 0.966, time 4.4 sec
epoch 6, train loss 0.0904, train acc 0.973, test loss 0.1007, test acc 0.971, time 5.4 sec
epoch 7, train loss 0.0710, train acc 0.979, test loss 0.0927, test acc 0.972, time 4.4 sec
epoch 8, train loss 0.1234, train acc 0.965, test loss 0.1044, test acc 0.970, time 4.2 sec
epoch 9, train loss 0.0633, train acc 0.981, test loss 0.0907, test acc 0.973, time 4.2 sec
epoch 10, train loss 0.0492, train acc 0.985, test loss 0.0875, test acc 0.976, time 4.3 sec
epoch 11, train loss 0.0391, train acc 0.988, test loss 0.08

## Exercícios

1. Qual impacto de alterar a probabilidade da camada de *Dropout*?  Como fica uma rede com probabilidade de 50% de "desligar" os neurônios?
1. E em relação ao valor de $\lambda$ para o *Weight Decay*, qual impacto? Teste valores como 0.0005 e 0.0001.
1. Qual efeito da quantidade de *epochs* na acurácia final do modelo?


